# 4. Extending SPARQL

This module is about custom extensions to SPARQL. Some are widely used, some only locally.

Before training with this module, a revision of Modules 3 should be done first.

---

The SPARQL language allows for the definition of custom functions. While in-built functions use keywords such as:

`FILTER (CONTAINS(...))`

using both `CONTAINS`, custom functions need to define IRIs for their functions:

`FILTER (ex:myfunc(...))`

* 4.1: A widely used SPARQL function extension is [GeoSPARQL](http://www.opengis.net/doc/IS/geosparql/1.1).

* 4.2: [RDFLib Time functions](https://github.com/RDFLib/timefuncs) are not widely used but are a good demonstration of how to implement SPARQL extension functions in an RDF toolkit.

* 4.3: Extension functions to leverage text indexes are implemented by all modern RDF DBs that support SPARQL queries.

### 4.1. GeoSPARQL

The [GeoSPARQL Standard](http://www.opengis.net/doc/IS/geosparql/1.1) defines toplogical and other functions, e.g.:

'Simple Features _Within)_' function is `<http://www.opengis.net/def/function/geosparql/sfWithin>` or `geof:sfWithin`.

This query will return all the Features, `?f`, whos Geometry's coordinates are within "POLYGON((...))".


```turtle
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>

SELECT *

WHERE {
    ?f 
       a geo:Feature ;
       schema:label ?l ;
       geo:hasGeometry/geo:asWKT ?wkt ;
    .

    FILTER STRSTARTS(?l, "N")
    
    FILTER geof:sfWithin(?wkt, "POLYGON((...))")
}
```

All GeoSAPRQL examples: <https://docs.ogc.org/is/22-047r1/22-047r1.html#_7bbe8aba-08ea-4910-bca1-bfaa79c79a71>

### 4.3 Time functions

The time functions library implements custom SPARQL functions in RDFLib for temporal relations: the temporal equivalent to GeoSAPRQL: <https://github.com/RDFLib/timefuncs>.

In [ ]:
from rdflib import Graph, Namespace
from rdflib.namespace import TIME
from timefuncs import is_after, is_before


def timefuncs_print(rdf, query, relation="is before"):
    def iri_clean(s):
        return str(s).replace("http://example.com/", "")
        
    g = Graph().parse(data=rdf_data, format="turtle")
    
    for r in g.query(
        q,
        initNs={
            "time": TIME, 
            "tfun": TFUN, 
            "ex": Namespace("http://example.com/")
        }):
        print(iri_clean(r[0]), relation, iri_clean(r[1]))
    
    
TFUN = Namespace("https://w3id.org/timefuncs/")

rdf_data = \
    """
    PREFIX time: <http://www.w3.org/2006/time#>
    PREFIX : <http://example.com/>
    
    :a
        a time:TemporalEntity ;
        time:inXSDDateTimeStamp "2021-07-15T23:59:59Z" ;
    .
    
    :b
        a time:TemporalEntity ;
        time:inXSDDateTimeStamp "2021-07-16T23:59:59Z" ;
    .
    
    :c
        a time:TemporalEntity ;
        time:inXSDDateTimeStamp "2021-07-17T23:59:59Z" ;
    .
        
    :d
        a time:TemporalEntity ;
        time:inXSDDateTimeStamp "2021-07-10T23:59:59Z" ;
    .
    """



q = """
     SELECT ?a ?b
     WHERE {
         ?a a time:TemporalEntity .
         ?b a time:TemporalEntity .

         FILTER tfun:isBefore(?a, ?b)
     }
     """

timefuncs_print(rdf_data, q)

In [ ]:
q = """
     SELECT ?a ?b
     WHERE {
         ?a a time:TemporalEntity .
         ?b a time:TemporalEntity .

         FILTER tfun:isAfter(?a, ?b)
     }
     """

timefuncs_print(rdf_data, q, relation="is after")

### 4.2. Lucene Text Index

RDF DBs implement graph indexes and can implement others too, such as spatial (using GeoSPARQL / GDAL) and (full) text indexes with tools like the well-known text indexer [Lucene](https://lucene.apache.org/).

This means you don't need to do this:

```sparql
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?a ?lbl
WHERE {
  GRAPH ?g {
  	?a rdfs:label ?lbl .
    
    FILTER STRSTARTS(?lbl, "Yundah St")
  }
}
```

Do this:

```sparql
PREFIX  text: <http://jena.apache.org/text#>

SELECT DISTINCT ?a ?lbl
WHERE {
  GRAPH ?g {
  	( ?a ?score ?lbl ?graph ) text:query "Yundah St*"
  }
}
ORDER BY DESC(?score)
LIMIT 5
```

Indexes other than graph indexes are custom per RDF DB product. Jena's full text index documentation is at <https://jena.apache.org/documentation/query/text-query.html>.